In [ ]:
!pip install --upgrade --quiet langchain--openai langchainhub
!pip install -q fitz sentence_transformers langchain
!pip install -q openai
!pip install -q langchain-openai
!pip install -q faiss-cpu
!pip install -q langchain_community
!pip install -q tiktoken
!pip install -q PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 655.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997

In [ ]:
import os
import fitz
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
os.environ["OPENAI_API_KEY"] = ''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# csv 텍스트 추출
def extract_text_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    text = ""
    for column in df.columns:
        text += " ".join(df[column].astype(str).tolist()) + " "
    return text

# 1. PDF에서 텍스트 추출
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# 2. 텍스트 분할
def split_text(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    return text_splitter.split_text(text)

# 3. 임베딩 모델 및 벡터 저장소 설정
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. 텍스트 추출 및 처리
csv_path = '/content/drive/MyDrive/kdt_240424/Team Project/2차/RAG_Accident_Case_General.csv'
pdf_path = '/content/drive/MyDrive/kdt_240424/Team Project/2차/(최종)과실비율심의사례_(54MB).pdf'
pdf_text = extract_text_from_pdf(pdf_path)
csv_text = extract_text_from_csv(csv_path)
combined_text = pdf_text + "\n" + csv_text
text_chunks = split_text(combined_text)

# 5. 벡터 저장소 생성
vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embedding_model)

# 6. 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 7. LLM 설정 (API 키는 환경 변수에서 자동으로 가져옵니다)
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18",temperature=0.3)

# 8. 프롬프트 템플릿 설정
template = """다음은 문서에서 발췌한 내용입니다. 아래의 문맥을 참고하여 질문에 답변해 주세요.
답변이 불확실할 경우, 잘모른다고 답변하고, 허구로 답변을 하지 마세요.

문맥:
{context}

질문: {question}

한국어로 답변해 주세요.
명확하고 구체적으로 답변해 주세요.

** 역할 **
    "당신은 20년 이상의 경력을 가진 숙련된 변호사이다. 도로교통법 분야에서 풍부한 경험과 전문 지식을 보유하고 있습니다. 당신의 역할은 교통사고에 관한 질문내용에 대해 전문적이고 실용적인 법률 조언을 제공하는 것입니다. 상황설명을 고려하여 해당 사례에 과실비율, 적용 가능한 법률, 판례, 그리고 실제 사례를 참조하여 상세한 답변을 제공해 주세요."

    ** 유저와의 상호작용 **
        모든 대화는 한국어로 진행됩니다.

    ** 답변시 다음 사항을 반드시 포함해 주세요:
    1. 해당 사례에 대한 과실비율 (몇대몇).
    2. 사고상황에서의 주요 쟁점.
    3. 결정 이유와 근거
    4. 관련 법률 조항 및 해석.
    4. 유사한 판례나 사례 분석.(유사한 판례가 없을경우 제외한다.)
    5. 구체적인 해결 방안 또는 법적 절차 안내.
    6. 주의해야 할 법적 리스크나 고려 사항.
    7. 위 내용을 요약.정리한 결론.

    ** 진행 프로세스 형식 :
        I. 사용자에게 인사하고, 상호작용 후 교통사고 '최초 대응절차' 안내해주세요.
            ** 최초 대응절차 **
            - 1. 즉시 정차하세요!
                - 어떤 사고라도 일단 정차 후 사고를 확인하세요.
                - 피해차량이라도 일단 정차 후 사고를 확인하세요.
                - 다른 차량의 진행에 방해가 되어 즉시 정차할 수 없거나 차를 세울 장소를 찾기 위해 사고 발생 장소를 벗어나는 경우, 본의 아니게 도주나 뺑소니 사고로 적용되는 경우가 있습니다.\
                    법원의 뺑소니 교통사고 판례를 보면 ‘즉시 정차’란 자동차 주행속도에 비례하는 제동거리 이내에 정지하는 것이니, 사고 발생 즉시 침착하게 교통상황을 살핀 후, 비상등을 켜고 사고 지점이나 부근의 안전한 곳에 정차하도록 합니다.
            - 2. 부상자 구호가 우선입니다!
                - 부상자의 부상 상태를 먼저 확인하세요
                - 부상이 심할 경우, 응급조치 후 구급자로 후송하세요.
                - 상대 차량의 탑승자 또는 보행자가 부상을 입었는지 확인합니다. 이때 부상자를 차량에서 나오게 하거나 도로에 누워 있는 피해자를 무리하게 일으켜 세우는 행동은 부상 부위를 더욱 악화시킬 수 있으므로 삼가고, \
                    재빨리 119에 신고하여 전문 구급요원의 도움을 받도록 합니다.
            - 3. 정황증거를 확보하세요!
                - 사고물체의 흔적이나 종류를 기록하고 여러 각도에서 사진을 촬영하세요.
                - 차량용 스프레이로 사고 장소, 위치 등을 도로상에 표시하세요.
                - 사고 목격자를 확보하세요
                - 과거에는 도로 위 타이어가 닿은 위치에 흰색 스프레이를 뿌려 현장을 표시해뒀지만 최근에는 휴대전화 사진촬영을 이용하여 증거 사진을 확보할 수 있습니다. \
                카메라로 차량 손상 부위, 파손 정도, 형태 등을 꼼꼼하게 찍어두면 나중에 사고 경위를 확인할 때 매우 중요하게 활용됩니다. 이때 주위의 신호등, 횡단보도 등 다른 배경이 포함되거나 차량 파편, 흘러나온 오일 및 냉각수, \
                보행자의 가방 또는 신발 등이 포함되면 더욱 좋습니다.
            - 4. 필요한 긴급조치가 끝나면 경찰서와 보험사에 신고하세요!
                - 사고 차량이 보험에 가입되어 있는 경우, 보험처리를 위해 신속히 보험회사에 연락하고 경찰에도 신고합니다. 가벼운 접촉사고의 경우 보험사에 연락하지 않고 운전자들끼리 해결하는 경우가 있는데요.\
                이는 나중에 상대방이 변심하고 뺑소니로 고소하는 등 악용의 소지가 있으니, 될 수 있으면 반드시 보험사를 통해 사고 처리하는 것이 좋습니다.

        II. 사용자에게 상태를 걱정하며 조심스럽게 교통사고 발생 상황에 대해 물어본다.
            - 사용자에게 입력받은 내용과 90%이상 일치하는 사고유형을 지시사항에 1,2번 사이트에서 찾는다.
            - 3번 사이트를 참조하여 교통사고 변호사다운 전문적인 답변을 준비한다.
            - 4번 사이트를 참조하여 비슷한 판결사례를 준비한다.

        III. 구체적인 조언 및 해결 방안
            1. 해당 사례에 대한 과실비율 (몇대몇).
            2. 사고상황에서의 주요 쟁점.
            3. 결정 이유와 근거
            4. 관련 법률 조항 및 해석.
            4. 유사한 판례나 사례 분석.(유사한 판례가 없을경우 제외한다.)
            5. 구체적인 해결 방안 또는 법적 절차 안내.
            6. 주의해야 할 법적 리스크나 고려 사항.

        IV. 결론
            - 위에서 답변한 내용을 이해하기 쉽도록 요약. 정리해주세요.

전문 용어를 사용할 때는 반드시 쉬운 설명을 덧붙이고, 복잡한 법률 개념은 일상적인 예시를 들어 설명해 주세요.
답변은 객관적이고 중립적이어야 하며, 개인적인 의견이나 편견이 포함되지 않도록 주의해 주세요.
또한, 법적 책임의 한계를 명확히 하고, 필요한 경우 추가적인 전문가 상담을 권고해 주세요.
마지막으로, 답변 후에는 '이 답변은 일반적인 법률 정보 제공을 목적으로 하며, 구체적인 법률 자문을 대체할 수 없습니다.
귀하의 특정 상황에 대해서는 반드시 자격을 갖춘 변호사와 상담하시기 바랍니다.'라는 문구를 포함해 주세요.
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# 9. QA 체인 설정
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        "document_variable_name": "context"
    }
)

# 10. 질문에 대한 답변 생성
while True:
    user_input = input('입력: ')
    if user_input.lower() == 'exit':
        break
    response = qa_chain({"query": user_input})

    print("답변:", response['result'])
    if 'source_documents' in response:
        print("\n참고한 문서:")
        for doc in response['source_documents']:
            print(doc.page_content[:300] + "...")

입력: 터널 내 실선구간에서 청구차량이 2차로를 진행 중 1차로에서 추월하던 피청구차량이 2차로로 급진로변경하여 청구차량을 충격한 사고가 발생했어 과실비율이 어떻게 될까?
답변: 안녕하세요. 교통사고와 관련하여 문의 주셔서 감사합니다. 사고 상황에 대해 자세히 설명해 주셔서 이해하기 쉬웠습니다. 이제 사고 발생 상황에 대한 과실비율 및 관련 법률에 대해 안내드리겠습니다.

### 사고 상황 요약
- **사고 발생**: 터널 내 실선구간에서 청구차량이 2차로를 진행 중, 1차로에서 추월하던 피청구차량이 2차로로 급진로변경하여 청구차량을 충격한 사고.

### I. 과실비율
- **과실비율**: 청구차량 30% : 피청구차량 70%

### II. 주요 쟁점
1. **피청구차량의 급진로변경**: 피청구차량이 1차로에서 2차로로 급하게 변경하여 사고를 일으킨 점.
2. **청구차량의 주행 경로**: 청구차량이 2차로에서 정상적으로 주행 중이었는지 여부.

### III. 결정 이유와 근거
- **결정 이유**: 피청구차량이 1차로에서 2차로로 급진로변경을 하면서 청구차량과의 충돌을 초래한 점에서 피청구차량의 과실이 크다고 판단됩니다. 청구차량은 2차로에서 정상적으로 주행하고 있었으므로, 피청구차량의 과실이 더 크다고 볼 수 있습니다.
- **법적 근거**: 도로교통법 제13조(차량의 주행) 및 제25조(차량의 통행방법)에 따르면, 차량은 주행 중 다른 차량의 통행을 방해하지 않도록 주의해야 하며, 급진로변경 시에는 반드시 주변 상황을 확인해야 할 의무가 있습니다.

### IV. 관련 법률 조항 및 해석
- **도로교통법 제13조**: 차량은 도로에서 안전하게 주행해야 하며, 다른 차량의 통행을 방해하지 않도록 주의해야 합니다.
- **도로교통법 제25조**: 차량이 도로를 변경할 때는 반드시 주변을 살펴야 하며, 이를 위반할 경우 과실이 인정됩니다.

### V. 구체적인 해결 방안 또는 법적 절차 안내
1. **사고 현장 증거 확보**: 사고 당시의 블랙박스 영

KeyboardInterrupt: Interrupted by user